<a href="https://colab.research.google.com/github/patrikrac/NLP_SQuAD2.0/blob/main/ATML_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Topics in Machine Learning - Natural Language Processing -- Group Assignment
> Students
> - Patrik Rác
> - Méline Trochon
> - Valentina Moretti
> - Mattia Colbertaldo

---



In [1]:
! pip install datasets transformers accelerate -U
! pip install --upgrade gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
# Imports
import os
import torch
import pandas

import matplotlib.pyplot as plt

from datasets import load_dataset, load_metric

## 1. Investigate Dataset


We will investigate the dataset ***SQuAD2.0 (Stanford Question Answering Dataset)*** avaliable [here](https://rajpurkar.github.io/SQuAD-explorer/).

In [3]:
# Set the directory to work in
WORKING_DIR = "./squad"
DATA_DIR = "./data"

In [4]:
# Download the dataset
print("Downloading the DEV dataset of SQuAD2.0")
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2024-01-05 16:36:58--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       100%[===================>]   4.17M  17.8MB/s    in 0.2s    

2024-01-05 16:36:59 (17.8 MB/s) - ‘dev-v2.0.json’ saved [4370528/4370528]



In [5]:
import pandas
dev_dataframe = pandas.read_json("./dev-v2.0.json")
print(f"Size of the dataset: {dev_dataframe.size} (e.g. Categories of questions)")
dev_dataframe.head()

Size of the dataset: 70 (e.g. Categories of questions)


,version,data
0,v2.0,"{'title': 'Normans', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Computational_complexity_theory', '..."
2,v2.0,"{'title': 'Southern_California', 'paragraphs':..."
3,v2.0,"{'title': 'Sky_(United_Kingdom)', 'paragraphs'..."
4,v2.0,"{'title': 'Victoria_(Australia)', 'paragraphs'..."


In [6]:
def count_paragraphs(row):
    return len(row["data"]["paragraphs"])

print(f"Number of individual paragraphs: {sum(dev_dataframe.apply(count_paragraphs, axis=1))}")


def count_questions(row):
  return sum([len(p["qas"]) for p in row["data"]["paragraphs"]])

print(f"Number of individual questions: {sum(dev_dataframe.apply(count_questions, axis=1))}")

def count_impossible_questions(row):
  n_impossible = 0
  for p in row["data"]["paragraphs"]:
    for q in p["qas"]:
      if q["is_impossible"]:
        n_impossible += 1
  return n_impossible


print(f"Number of impossible questions: {sum(dev_dataframe.apply(count_impossible_questions, axis=1))}")

Number of individual paragraphs: 1204
Number of individual questions: 11873
Number of impossible questions: 5945


In [7]:
print(type(dev_dataframe.iloc[0]["data"]))

import json
print(json.dumps(dev_dataframe.iloc[1]["data"], sort_keys=False, indent=4))

<class 'dict'>
{
    "title": "Computational_complexity_theory",
    "paragraphs": [
        {
            "qas": [
                {
                    "question": "What branch of theoretical computer science deals with broadly classifying computational problems by difficulty and class of relationship?",
                    "id": "56e16182e3433e1400422e28",
                    "answers": [
                        {
                            "text": "Computational complexity theory",
                            "answer_start": 0
                        },
                        {
                            "text": "Computational complexity theory",
                            "answer_start": 0
                        },
                        {
                            "text": "Computational complexity theory",
                            "answer_start": 0
                        }
                    ],
                    "is_impossible": false
                },
           

In [8]:
# Example of a Paragraph and a question
import textwrap
print("Category of given paragraph: {}\n".format(dev_dataframe.iloc[0]["data"]["title"]))

example_datum = dev_dataframe.iloc[0]["data"]["paragraphs"][0]
print("Context\n {}\n".format(textwrap.fill(example_datum["context"], 50)))

print("Example Question:\n{}\n".format(example_datum["qas"][0]["question"]))

print("Example Answer:\n{}\n".format(example_datum["qas"][0]["answers"][0]["text"]))

Category of given paragraph: Normans

Context
 The Normans (Norman: Nourmands; French: Normands;
Latin: Normanni) were the people who in the 10th
and 11th centuries gave their name to Normandy, a
region in France. They were descended from Norse
("Norman" comes from "Norseman") raiders and
pirates from Denmark, Iceland and Norway who,
under their leader Rollo, agreed to swear fealty
to King Charles III of West Francia. Through
generations of assimilation and mixing with the
native Frankish and Roman-Gaulish populations,
their descendants would gradually merge with the
Carolingian-based cultures of West Francia. The
distinct cultural and ethnic identity of the
Normans emerged initially in the first half of the
10th century, and it continued to evolve over the
succeeding centuries.

Example Question:
In what country is Normandy located?

Example Answer:
France



In [9]:
# We will now create a list with pairs (Title, Question, Paragraph, Answers)
data_list = list()
categories = list()
paragraphs = list()
questions = list()

for _, row in dev_dataframe.iterrows():
  categories.append(row["data"]["title"])
  for p in row["data"]["paragraphs"]:
    paragraphs.append(p["context"])
    for q in p["qas"]:
      questions.append(q["question"])
      data_list.append((row["data"]["title"], q, p["context"], q["answers"]))

In [10]:
paragraph_len_dict = dict()
for p in paragraphs:
  lp = len(p)
  if lp not in paragraph_len_dict:
    paragraph_len_dict[lp] = 1
  else:
    paragraph_len_dict[lp] += 1

plt.bar(paragraph_len_dict.keys(), paragraph_len_dict.values(), align='center')
plt.xlabel("Number of characters")
plt.ylabel("Number of apperances")
#plt.savefig("len_distribution.png")
plt.show()

## TODO Section
---

- Investigate the data further (Possibly some plots etc.)
- Train Word2Vec on the corpus of paragraphs.
- (Potentially index the paragraphs) -> Find appropriate Datastructure potentially

=> Prepare Data for Training

---
- Investigate the application of `DistilBert`
  - Fine Tuning
  - One-Shot learning
  - (Two-Shot learning)
- Come up with alternatives (Custom Transformer etc.)
---


## Training Word2Vec

In [14]:
# Load the nessecary libraries
import re

# remove newline characters
paragraphs_w2v = [re.sub('\n', ' ', p) for p in paragraphs]
# remove email addresses
paragraphs_w2v = [re.sub('[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '', p) for p in paragraphs_w2v]
# split sentences
paragraph_sentences = [re.split('[?!.]\s', p) for p in paragraphs_w2v]
paragraph_sentences[:3]

[['The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France',
  'They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia',
  'Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia',
  'The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'],
 ['The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East',
  'The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Ca

In [15]:
# Flatten the sentences using the flatten function from pandas in order to
from pandas.core.common import flatten
paragraph_sentences = list(flatten(paragraph_sentences))
paragraph_sentences[:20]

['The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France',
 'They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia',
 'Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia',
 'The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East',
 'The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic 

In [21]:
# Preprocess the sentences (very simple tokenization)
paragraph_tokenized_sentences = [re.sub('\W', ' ', sentence).lower().split() for sentence in paragraph_sentences]
# remove sentences that are only 1 word long
paragraph_tokenized_sentences = [sentence for sentence in paragraph_tokenized_sentences if len(sentence) > 1]

for sentence in paragraph_tokenized_sentences[:10]:
    print(sentence)
print()
print(f"Number of tokenized sentences = {len(paragraph_tokenized_sentences)}")

['the', 'normans', 'norman', 'nourmands', 'french', 'normands', 'latin', 'normanni', 'were', 'the', 'people', 'who', 'in', 'the', '10th', 'and', '11th', 'centuries', 'gave', 'their', 'name', 'to', 'normandy', 'a', 'region', 'in', 'france']
['they', 'were', 'descended', 'from', 'norse', 'norman', 'comes', 'from', 'norseman', 'raiders', 'and', 'pirates', 'from', 'denmark', 'iceland', 'and', 'norway', 'who', 'under', 'their', 'leader', 'rollo', 'agreed', 'to', 'swear', 'fealty', 'to', 'king', 'charles', 'iii', 'of', 'west', 'francia']
['through', 'generations', 'of', 'assimilation', 'and', 'mixing', 'with', 'the', 'native', 'frankish', 'and', 'roman', 'gaulish', 'populations', 'their', 'descendants', 'would', 'gradually', 'merge', 'with', 'the', 'carolingian', 'based', 'cultures', 'of', 'west', 'francia']
['the', 'distinct', 'cultural', 'and', 'ethnic', 'identity', 'of', 'the', 'normans', 'emerged', 'initially', 'in', 'the', 'first', 'half', 'of', 'the', '10th', 'century', 'and', 'it', 'c

In [23]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(paragraph_tokenized_sentences, vector_size=30, min_count=5, window=10)

len(model.wv)

3998

### Inspection of the W2V embedding

In [24]:
term = 'car'
model.wv[term]

array([ 0.04057898, -0.00312986,  0.17986928,  0.07059216,  0.04381702,
        0.08370378,  0.22301102,  0.03001559, -0.11262328,  0.0514932 ,
        0.09641131,  0.0892488 , -0.04129973, -0.14210753, -0.00300549,
       -0.11222779,  0.00609669, -0.07083324, -0.22837777, -0.0841409 ,
        0.03065133,  0.00296772,  0.02482023,  0.17426671,  0.09259891,
        0.05354922,  0.05035822,  0.09622449, -0.12108433, -0.20909923],
      dtype=float32)

In [29]:
term = 'king'
model.wv.most_similar(term)

[('with', 0.9987552165985107),
 ('from', 0.9986580610275269),
 ('on', 0.998618483543396),
 ('american', 0.9985692501068115),
 ('through', 0.9985644817352295),
 ('study', 0.9984967708587646),
 ('modern', 0.9984954595565796),
 ('upper', 0.998436450958252),
 ('national', 0.99842768907547),
 ('de', 0.9984264969825745)]

In [28]:
vec = model.wv['king'] + (model.wv['woman'] - model.wv['man'])
model.wv.similar_by_vector(vec)

[('england', 0.9981468319892883),
 ('american', 0.9981021285057068),
 ('architecture', 0.9980552196502686),
 ('king', 0.9980142712593079),
 ('chinese', 0.9977939128875732),
 ('building', 0.9977544546127319),
 ('under', 0.9977461695671082),
 ('black', 0.9977372288703918),
 ('his', 0.9977056980133057),
 ('price', 0.9976696372032166)]

## Finetuning DistilBERT

In [11]:
# Import the transformers package (We'll be taking a pretrained model from here)
!pip install transformers
import transformers
# print(transformers.__version__)

We define the pretrained model we want to use. In this case we are using the _base_ _uncased_ variant of ***DistilBert***. Additionally we are fixing the batch size.

In [12]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

Load the dataset. Here we can go down two distinct routes that will not have a particularly large impact on the next steps.
1. Use the already prepared ***SQuAD2.0*** dataset from huggingface using the _load_dataset_ method.
2. Transform the prepared data from above into a similar dataset class

The first step is significantly easier and is therefore used, however, even with the second approach there are no major difficulties as the data is already present in a prepared format. We could then just use the already predefined split or use the huggingface funciton _train_test_split()_ in order to split the data.

For the first approach we additionally reduce the size of the training set as we want to achieve the fine tuning in a reasonable time frame. We thus select a random (fixed seed) subset of 20_000 items.

In [13]:
squad2_datasets = load_dataset("squad_v2") # Possibly replace the huggingface prepared dataset with the custom loaded one...
print(squad2_datasets)
# Reduce the size of the training data set by selecting a random subsample
squad2_datasets["train"] = squad2_datasets["train"].shuffle(seed=42).select(range(20000))
print(squad2_datasets)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})


We then import the tokenizer that was used to pretrain the given model as we will also use it. For this reason we import the _AutoTokenizer_ class from _transformers_ and use its method _from_pretrained_.

In [42]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


We can now simply test the tokenizer on a sample question/paragrpah and see what happens!

In [58]:
sample_IDX = 1
switched_vocab = {v: k for k, v in tokenizer.vocab.items()}

# Example with a Question
sample_question = squad2_datasets["train"][sample_IDX]["question"]
print(f"The sample question is: ''{sample_question}''")
print(f"The tokenized question is: {tokenizer(sample_question).input_ids}")
print('token values: ', [switched_vocab[i] for i in tokenizer(sample_question).input_ids])
print()

# Example with a Paragraph
sample_paragraph = squad2_datasets["train"][sample_IDX]["context"]
print(f"The sample context is: ''{sample_paragraph}''")
print(f"The tokenized context is: {tokenizer(sample_paragraph).input_ids}")
print('token values: ', [switched_vocab[i] for i in tokenizer(sample_paragraph).input_ids])
print()

# Complete example (Question + Context)
tokenization_example_complete = tokenizer(sample_question, sample_paragraph)
print(f"The tokenized example is: {tokenization_example_complete.input_ids}")
print('token values: ', [switched_vocab[i] for i in tokenization_example_complete.input_ids])
print()

print("Vocabulary size: ", len(tokenizer.vocab))

The sample question is: ''What was Alexandria known for?''
The tokenized question is: [101, 2054, 2001, 10297, 2124, 2005, 1029, 102]
token values:  ['[CLS]', 'what', 'was', 'alexandria', 'known', 'for', '?', '[SEP]']

The sample context is: ''Alexandria was the most important trade center in the whole empire during Athanasius's boyhood. Intellectually, morally, and politically—it epitomized the ethnically diverse Graeco-Roman world, even more than Rome or Constantinople, Antioch or Marseilles. Its famous catechetical school, while sacrificing none of its famous passion for orthodoxy since the days of Pantaenus, Clement of Alexandria, Origen of Alexandria, Dionysius and Theognostus, had begun to take on an almost secular character in the comprehensiveness of its interests, and had counted influential pagans among its serious auditors.''
The tokenized context is: [101, 10297, 2001, 1996, 2087, 2590, 3119, 2415, 1999, 1996, 2878, 3400, 2076, 2012, 15788, 24721, 1005, 1055, 2879, 9021, 10

In [16]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [17]:
pad_on_right = tokenizer.padding_side == "right"

In [18]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [19]:
from datasets import load_dataset
tokenized_datasets = squad2_datasets.map(prepare_train_features, batched=True, remove_columns=squad2_datasets["train"].column_names)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

### Actual Fine Tuning of the Model

In [20]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [22]:
from transformers import default_data_collator

data_collator = default_data_collator

In [23]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.101000,1.668057
2,1.485700,1.448959


Epoch,Training Loss,Validation Loss
1,2.101000,1.668057
2,1.485700,1.448959
3,1.228000,1.479264


TrainOutput(global_step=3792, training_loss=1.7248736675278546, metrics={'train_runtime': 2684.7761, 'train_samples_per_second': 22.595, 'train_steps_per_second': 1.412, 'total_flos': 5944356755430912.0, 'train_loss': 1.7248736675278546, 'epoch': 3.0})

In [25]:
trainer.save_model("test-squad-trained")

##### Evaluation
---

In [26]:
for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [27]:
# TODO: Evaluation. The evaluation of the model should be rather genereal and work with all model training strategies.
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [28]:
validation_features = squad2_datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=squad2_datasets["validation"].column_names
)

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [29]:
raw_predictions = trainer.predict(validation_features)

In [30]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [31]:
import numpy as np
n_best_size = 20
max_answer_length = 30
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = squad2_datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 11.796044, 'text': 'France'},
 {'score': 7.748126,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway'},
 {'score': 6.9550233,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland'},
 {'score': 6.293585,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark'},
 {'score': 5.1805305, 'text': 'region in France'},
 {'score': 5.0064774, 'text': 'a region in France'},
 {'score': 4.70089, 'text': 'in France'},
 {'score': 4.6213884, 'text': 'France.'},
 {'score': 3.965252, 'text': 'Normandy, a region in France'},
 {'score': 3.8346362,
  'text': 'French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': 3.7472296,
  'text': 'France. They were descended from Norse ("Norman" comes f

In [35]:
# Code for computing the score of the model

from tqdm.auto import tqdm
import collections

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [36]:
final_predictions = postprocess_qa_predictions(squad2_datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 11873 example predictions split into 12134 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

In [38]:
metric = load_metric("squad_v2")

<ipython-input-38-9cd7db57dbcb>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [41]:
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in squad2_datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 54.703950138970775,
 'f1': 58.20276713732525,
 'total': 11873,
 'HasAns_exact': 52.02429149797571,
 'HasAns_f1': 59.03195921414672,
 'HasAns_total': 5928,
 'NoAns_exact': 57.37594617325484,
 'NoAns_f1': 57.37594617325484,
 'NoAns_total': 5945,
 'best_exact': 55.08296134085741,
 'best_exact_thresh': 0.0,
 'best_f1': 58.32510565770994,
 'best_f1_thresh': 0.0}


{'exact': 54.703950138970775,
 'f1': 58.20276713732525,
 'total': 11873,
 'HasAns_exact': 52.02429149797571,
 'HasAns_f1': 59.03195921414672,
 'HasAns_total': 5928,
 'NoAns_exact': 57.37594617325484,
 'NoAns_f1': 57.37594617325484,
 'NoAns_total': 5945,
 'best_exact': 55.08296134085741,
 'best_exact_thresh': 0.0,
 'best_f1': 58.32510565770994,
 'best_f1_thresh': 0.0}
